In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df.shape

(768, 9)

In [9]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [11]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [13]:
X.shape

(768, 8)

In [15]:
y.shape

(768,)

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [19]:
X = scaler.fit_transform(X)

In [21]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [23]:
X.shape

(768, 8)

In [25]:
y.shape

(768,)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [154]:
import tensorflow 
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [156]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5936 - loss: 0.7017 - val_accuracy: 0.5779 - val_loss: 0.6983
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.6721 - loss: 0.6426 - val_accuracy: 0.6234 - val_loss: 0.6446
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 838us/step - accuracy: 0.6801 - loss: 0.6084 - val_accuracy: 0.6688 - val_loss: 0.6082
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - accuracy: 0.7023 - loss: 0.5749 - val_accuracy: 0.7143 - val_loss: 0.5793
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - accuracy: 0.7371 - loss: 0.5365 - val_accuracy: 0.7338 - val_loss: 0.5567
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.7335 - loss: 0.5565 - val_accuracy: 0.7403 - val_loss: 0.5391
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - accuracy: 0.7429 - loss: 0.5286 - val_accuracy: 0.7403 - val_loss: 0.5262
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.7609 - loss: 0.5109 - val_accuracy: 0.74

In [36]:
import keras_tuner as kt

In [48]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
    return model

In [54]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [55]:
tuner.search(X_train, y_train,epochs=5,validation_data=(X_test, y_test))

In [56]:
print(tuner.get_best_hyperparameters()[0].values)

{'optimizer': 'adam'}


In [57]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [67]:
model=tuner.get_best_models(num_models=1)[0]

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [69]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [71]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, batch_size=32, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7528 - loss: 0.5252 - val_accuracy: 0.7792 - val_loss: 0.5191
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - accuracy: 0.7254 - loss: 0.5100 - val_accuracy: 0.7792 - val_loss: 0.5075
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.7468 - loss: 0.5079 - val_accuracy: 0.7857 - val_loss: 0.4989
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - accuracy: 0.7635 - loss: 0.4904 - val_accuracy: 0.7792 - val_loss: 0.4923
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - accuracy: 0.7470 - loss: 0.5002 - val_accuracy: 0.7792 - val_loss: 0.4867
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.7625 - loss: 0.4795 - val_accuracy: 0.7857 - val_loss: 0.4815
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - accuracy: 0.7822 - loss: 0.4742 - val_accuracy: 0.7792 - val_loss: 0.4782
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.7802 - loss: 0.4698 - val_a

In [95]:
def build_model(hp):
    model=Sequential()
    units = hp.Int('units', min_value = 8, max_value = 128, step=8)
    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [109]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='dir2/', project_name='srk')

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [111]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 06s


In [113]:
tuner.get_best_hyperparameters()[0].values

{'units': 112}

In [115]:
model=tuner.get_best_models(num_models=1)[0]

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [117]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7542 - loss: 0.4843 - val_accuracy: 0.7857 - val_loss: 0.4881
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - accuracy: 0.7685 - loss: 0.4655 - val_accuracy: 0.7792 - val_loss: 0.4842
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - accuracy: 0.7626 - loss: 0.4641 - val_accuracy: 0.7792 - val_loss: 0.4828
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - accuracy: 0.7565 - loss: 0.4647 - val_accuracy: 0.7792 - val_loss: 0.4821
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - accuracy: 0.7861 - loss: 0.4346 - val_accuracy: 0.7727 - val_loss: 0.4805
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - accuracy: 0.7875 - loss: 0.4555 - val_accuracy: 0.7727 - val_loss: 0.4799
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - accuracy: 0.7741 - loss: 0.4539 - val_accuracy: 0.7857 - val_loss: 0.4782
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.7825 - loss: 0.4454 - val_a

Final

In [164]:
def build_model(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layer', min_value=1, max_value=10)):
        if counter == 0:
            model.add(
                Dense(
                    hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
                    input_dim=8
                )
            )
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(
                Dense(
                    hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
                )
            )
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter += 1
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd', 'nadam', 'adadelta']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [166]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='final_dir',
    project_name='final_pro'
)

Reloading Tuner from final_dir/final_pro/tuner0.json


In [168]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [178]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 7,
 'units0': 120,
 'activation0': 'sigmoid',
 'optimizer': 'adam',
 'units1': 40,
 'activation1': 'sigmoid',
 'units2': 88,
 'activation2': 'sigmoid',
 'units3': 96,
 'activation3': 'sigmoid',
 'units4': 8,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'relu',
 'units6': 8,
 'activation6': 'relu',
 'dropout0': 0.1,
 'dropout1': 0.1,
 'dropout2': 0.1,
 'dropout3': 0.1,
 'dropout4': 0.1,
 'dropout5': 0.1,
 'dropout6': 0.1}

In [172]:
model = tuner.get_best_models(num_models=1)[0]

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [173]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 120)            │         1,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         4,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 88)             │         3,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 88)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 96)             │         8,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,001 (74.22 KB)

 Trainable params: 19,001 (74.22 KB)

 Non-trainable params: 0 (0.00 B)

In [176]:
model.fit(X_train, y_train, epochs=200, batch_size=20, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6767 - loss: 0.6412 - val_accuracy: 0.6429 - val_loss: 0.6627
Epoch 8/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6418 - loss: 0.6679 - val_accuracy: 0.6429 - val_loss: 0.6538
Epoch 9/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6604 - loss: 0.6506 - val_accuracy: 0.6429 - val_loss: 0.6538
Epoch 10/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - accuracy: 0.6476 - loss: 0.6568 - val_accuracy: 0.6429 - val_loss: 0.6502
Epoch 11/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - accuracy: 0.6641 - loss: 0.6371 - val_accuracy: 0.6429 - val_loss: 0.6484
Epoch 12/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - accuracy: 0.6639 - loss: 0.6528 - val_accuracy: 0.6429 - val_loss: 0.6448
Epoch 13/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - accuracy: 0.6474 - loss: 0.6555 - val_accuracy: 0.6429 - val_loss: 0.6148
Epoch 14/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.6514 - loss: 0.6037 - val_accur